<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/CNN-intra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, Dense, AveragePooling1D, Flatten
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv('/content/drive/MyDrive/intravehicle.csv')

In [4]:
y_data = data.pop('4')
x_data = data

In [5]:
train_x,test_x,train_y,test_y = train_test_split(x_data, y_data, test_size=0.2, random_state=0, shuffle=True) 

In [6]:
train_x = train_x.reset_index(drop=True)
test_x = test_x.reset_index(drop=True) 
train_y = train_y.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)

In [7]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [8]:
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)

In [9]:
model = Sequential()
model.add(Conv1D(activation="sigmoid", input_shape=(len(x_data.columns),1), kernel_size=2, filters=12))
model.add(AveragePooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 3, 12)             36        
                                                                 
 average_pooling1d (AverageP  (None, 1, 12)            0         
 ooling1D)                                                       
                                                                 
 flatten (Flatten)           (None, 12)                0         
                                                                 
 dense (Dense)               (None, 4)                 52        
                                                                 
Total params: 88
Trainable params: 88
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(train_x, train_y, batch_size=32)

414237/414237 [==============================] - 1130s 3ms/step - loss: 0.2324 - accuracy: 0.8986


In [12]:
pred = model.predict(test_x, verbose=1)
pred_y = np.argmax(pred, axis=1)

103560/103560 [==============================] - 150s 1ms/step


In [13]:
confusion_matrix(test_y, pred_y)

array([[  14165,       0,  103562,       0],
       [      0,   47493,   50693,      16],
       [  10049,   21536, 2815796,     529],
       [      0,       0,  119226,  130830]])

In [15]:
report = classification_report(test_y, pred_y, digits=5)
print(report)

              precision    recall  f1-score   support

           0    0.58499   0.12032   0.19959    117727
           1    0.68802   0.48363   0.56799     98202
           2    0.91147   0.98872   0.94853   2847910
           3    0.99585   0.52320   0.68600    250056

    accuracy                        0.90778   3313895
   macro avg    0.79508   0.52897   0.60053   3313895
weighted avg    0.89962   0.90778   0.89084   3313895

